In [37]:
import numpy as np
import pandas as pd

from Correlaciones import ewma
from Curvas import seleccionar_bono_fecha, seleccionar_todos_bonos
from Pivotes import *

In [38]:
def leer_archivo(nombreArchivo):

    """
    Extrae un archivo excel a un dataframe, calculando su valor de retorno
    con la funcion retorno_bonos
    
    """

    archivo = pd.read_excel('C:\\Users\\groso\\Desktop\\Practica\\Intento\\ArchivosExcel\\'+ nombreArchivo)
    columnas = ["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]
    archivo = archivo[columnas]

    return archivo

leer_archivo("BSANTANDER.SN.xlsx")


,Date,Open,High,Low,Close,Adj Close,Volume
0,2011-06-01,40.450001,40.700001,40.400002,40.400002,28.522612,56108459
1,2011-06-02,40.400002,40.709999,40.110001,40.599998,28.663807,54130590
2,2011-06-03,40.630001,40.799999,40.500000,40.799999,28.805012,47355452
3,2011-06-06,40.799999,40.900002,40.680000,40.689999,28.727345,35107202
4,2011-06-07,40.900002,41.279999,40.900002,41.230000,29.108595,57176259
...,...,...,...,...,...,...,...
2134,2020-01-03,45.279999,45.279999,43.560001,45.279999,45.279999,29739554
2135,2020-01-06,46.000000,46.000000,44.490002,46.000000,46.000000,72818156
2136,2020-01-07,46.299999,46.299999,44.709999,46.299999,46.299999,5343453
2137,2020-01-08,46.200001,46.389999,45.509998,46.200001,46.200001,30273536


In [39]:
def retornos_acciones(nombreAccion):

    """
    Funcion encargada de calcular los retornos
    de una accion con la columna Adj Close
    :param nombreAccion: Nombre del archivo de la accion

    """

    tabla_accion = leer_archivo(nombreAccion)
    tabla_adj = tabla_accion["Adj Close"]
    fechas = tabla_accion["Date"]
    lenght = len(tabla_adj)
    arreglo = []

    for i in range(lenght):
        
        if i == 0:
            arreglo.append(0)

        else:
            valor = np.log(tabla_adj[i] / tabla_adj[i-1])
            arreglo.append(valor)

    df = pd.DataFrame({"Fecha": fechas, "Adj Close": tabla_adj, "Retornos": arreglo})
    return df

retornos = retornos_acciones("BSANTANDER.SN.xlsx")
retornos

,Fecha,Adj Close,Retornos
0,2011-06-01,28.522612,0.000000
1,2011-06-02,28.663807,0.004938
2,2011-06-03,28.805012,0.004914
3,2011-06-06,28.727345,-0.002700
4,2011-06-07,29.108595,0.013184
...,...,...,...
2134,2020-01-03,45.279999,0.017601
2135,2020-01-06,46.000000,0.015776
2136,2020-01-07,46.299999,0.006501
2137,2020-01-08,46.200001,-0.002162


In [40]:
def volatilidades_acciones(retornos_acciones):

    """
    Funcion que calcula las volatilidades de una accion
    dado los retornos de esta
    :param retorno_acciones: DataFrame con los retornos de una accion
    :return DataFrame con las volatilidades

    """

    final = []
    lenght = len(retornos_acciones)

    calculo = ewma(retornos_acciones, 0.94)
    final.append(calculo["Vol c/ajuste"][0])

    df = pd.DataFrame({"Volatilidad": final})

    return df

volatilidades_acciones(retornos["Retornos"])

,Volatilidad
0,0.017051


In [41]:
def calculo_varias_acciones(nombreAcciones):

    """
    Funcion encargada del calculo de retornos
    para muchas acciones
    :param nombreAcciones: Vector con el nombre de el archivo de las acciones
    :return DataFrame con los retornos de cada accion

    """

    lenght = len(nombreAcciones)
    df = pd.DataFrame()
    for i in range(lenght):

        retornos = retornos_acciones(nombreAcciones[i])
        df[str(nombreAcciones[i]).split(".")[0]] = retornos["Retornos"]


    return df

acciones = ["BSANTANDER.SN.xlsx", "ENTEL.SN.xlsx", "IANSA.SN.xlsx"]
retornos_acciones = calculo_varias_acciones(acciones)
retornos_acciones


,BSANTANDER,ENTEL,IANSA
0,0.000000,0.000000,0.000000
1,0.004938,-0.005686,0.008785
2,0.004914,0.000340,-0.016857
3,-0.002700,-0.004205,-0.011731
4,0.013184,0.003832,-0.016129
...,...,...,...
2134,0.017601,0.018349,0.000000
2135,0.015776,-0.005305,0.000000
2136,0.006501,0.005305,0.000000
2137,-0.002162,0.011389,0.000000


In [42]:
def calculo_volatilidades_acciones(dfRetornos):

    """
    Funcion encargada de calcular las volatilidades
    para todas las acciones con el DataFrame de los retornos
    :param dfRetornos: DataFrame con todos los retornos de las acciones
    :return DataFrame con las volatilidad de cada accion

    """

    lenght = dfRetornos.shape[1]
    columnas = list(dfRetornos)
    df = pd.DataFrame()
    archivo = leer_archivo("BSANTANDER.SN.xlsx")
    df["Empresa"] = columnas
    volatilidades = []

    for i in range(lenght):

        retornos = dfRetornos[columnas[i]]
        volatilidad = volatilidades_acciones(retornos)
        volatilidades.append(volatilidad.values[0][0]) 

    df["Volatilidades"] = volatilidades
    return df

volatilidades = calculo_volatilidades_acciones(retornos_acciones)
volatilidades

,Empresa,Volatilidades
0,BSANTANDER,0.017051
1,ENTEL,0.025222
2,IANSA,0.012328


In [43]:
def correlacion_acciones(dfRetornos, dfVolatilidades):

    """
    Funcion encargada de calcular la correlacion de las acciones
    :param dfRetornos: DataFrame con todos los retornos de las acciones
    :param dfVolatilidades: DataFrame con las volatilidades de las acciones
    :return DataFrame con la matriz de correlacion

    """

    num_acciones = len(dfVolatilidades)
    correlacion = ewma_new_new_pivotes(num_acciones, dfRetornos, dfVolatilidades)
    return correlacion

correlacion = correlacion_acciones(retornos_acciones, volatilidades["Volatilidades"] )

correlacion

,BSANTANDER,ENTEL,IANSA
BSANTANDER,0.017051,0.485747,0.000886
ENTEL,0.485747,0.025222,0.098124
IANSA,0.000886,0.098124,0.012328


In [44]:
def covarianza_acciones(dfRetornos, dfVolatilidades):

    """
    Funcion encargada de calcular la covarianza de las acciones
    :param dfRetornos: DataFrame con los retornos de todas las acciones
    :param dfVolatilidades: DataFrame con la volatilidad de las acciones
    :return DataFrame con la matriz de covarianza de las acciones

    """

    num_acciones = len(dfVolatilidades)
    covarianza = covarianza_pivotes(num_acciones, dfRetornos, dfVolatilidades)
    return covarianza

covarianza = covarianza_acciones(retornos_acciones, volatilidades["Volatilidades"])
covarianza

,BSANTANDER,ENTEL,IANSA
BSANTANDER,2.907342e-04,0.000209,1.862122e-07
ENTEL,2.089026e-04,0.000636,3.050953e-05
IANSA,1.862122e-07,0.000031,1.519679e-04
